In [2]:
from Crawler.Utils import get_and_unwrap
from Crawler.Data import SyllabusRegistry
from Crawler.Constants import Constants

In [3]:
html = get_and_unwrap(Constants.CMU_SYLLABUS_REGISTRY_URL.value)

In [ ]:
sr = SyllabusRegistry(html=html)

f24 = sr.semesters[0]
f24_archi = f24.departments[0]
f24_archi.get()
f24_archi_courses = f24_archi.courses

f24_archi_courses['Available Syllabi'][3].href
f24_archi_courses['Available Syllabi'][3].get()
f24_archi_courses['Available Syllabi'][3].html

## Sqlite

In [3]:
import pickle
pickle_data = pickle.dumps(sr)
loaded_sr = pickle.loads(pickle_data) 
loaded_sr.semesters[0].departments[0].courses['Available Syllabi'][3].archive

NameError: name 'sr' is not defined

In [1]:
import sqlite3
from Crawler.Data import ArchivedSemester, PDF, Webpage
from Crawler.Constants import Constants

conn = sqlite3.connect('syllabus_registry.db')
cursor = conn.cursor()

In [2]:
create_database_script = f'''
    CREATE TABLE semesters (
        semester_id INTEGER PRIMARY KEY AUTOINCREMENT, 
        name TEXT NOT NULL, 
        acronym TEXT, 
        href TEXT, 
        archived INTEGER NOT NULL
    );
    PRAGMA foreign_keys = ON;
    CREATE TABLE departments (
        department_id INTEGER PRIMARY KEY AUTOINCREMENT, 
        name TEXT NOT NULL, 
        acronym TEXT, 
        href TEXT NOT NULL, 
        semester_id INTEGER, 
        FOREIGN KEY (semester_id) REFERENCES semesters (semester_id) ON DELETE CASCADE
    );
    CREATE TABLE courses (
        course_id INTEGER PRIMARY KEY AUTOINCREMENT, 
        name TEXT NOT NULL, 
        acronym TEXT, 
        href TEXT NOT NULL, 
        department_id INTEGER NOT NULL, 
        category TEXT NOT NULL CHECK(category in ({', '.join("'" + s + "'" for s in Constants.COURSE_CATEGORIES.value)})),
        syllabus_category TEXT CHECK(syllabus_category in ('PDF', 'Webpage')), 
        syllabus_href TEXT, 
        syllabus_name TEXT, 
        FOREIGN KEY (department_id) REFERENCES departments (department_id) ON DELETE CASCADE
    );
'''
cursor.executescript(create_database_script)

In [4]:
cursor.execute("SELECT name, sql FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

In [5]:
tables

[('semesters',
  'CREATE TABLE semesters (\n        semester_id INTEGER PRIMARY KEY AUTOINCREMENT, \n        name TEXT NOT NULL, \n        acronym TEXT, \n        href TEXT, \n        archived INTEGER NOT NULL\n    )'),
 ('sqlite_sequence', 'CREATE TABLE sqlite_sequence(name,seq)'),
 ('departments',
  'CREATE TABLE departments (\n        department_id INTEGER PRIMARY KEY AUTOINCREMENT, \n        name TEXT NOT NULL, \n        acronym TEXT, \n        href TEXT NOT NULL, \n        semester_id INTEGER, \n        FOREIGN KEY (semester_id) REFERENCES semesters (semester_id) ON DELETE CASCADE\n    )'),
 ('courses',
  "CREATE TABLE courses (\n        course_id INTEGER PRIMARY KEY AUTOINCREMENT, \n        name TEXT NOT NULL, \n        acronym TEXT, \n        href TEXT NOT NULL, \n        department_id INTEGER NOT NULL, \n        category TEXT NOT NULL CHECK(category in ('Available Syllabi', 'Unavailable Syllabi', 'Individualized Experiences')),\n        syllabus_category TEXT CHECK(syllabus_cat

In [7]:
import pickle
with open('syllabus_registry.pkl', 'rb') as f:
    sr = pickle.load(f) 

In [ ]:
for s in sr.semesters:
    cursor.execute('''
        INSERT INTO semesters (name, acronym, href, archived) values (?, ?, ?, ?)
    ''', (s.name, s.acronym, s.href if isinstance(s, ArchivedSemester) else None, int(isinstance(s, ArchivedSemester))))
    conn.commit()
    semester_id = cursor.lastrowid
    for d in s.departments:
        cursor.execute('''
            INSERT INTO departments (name, acronym, href, semester_id) values (?, ?, ?, ?)
        ''', (d.name, d.acronym, d.href, semester_id))
        conn.commit()
        department_id = cursor.lastrowid
        if d.processed:
            assert d.courses is not None
            for c in d.courses['Available Syllabi']:
                if c.processed:
                    archive = c.archive
                    if archive is not None:
                        # If processing is unsuccessful, do not insert.
                        if isinstance(archive, PDF):
                            syllabus_category = 'PDF'
                            syllabus_href = archive.href
                            syllabus_name = archive.name
                        elif isinstance(archive, Webpage):
                            syllabus_category = 'Webpage'
                            syllabus_href = archive.href
                            syllabus_name = None
                        else:
                            # Unreachable.
                            breakpoint()
                        cursor.execute('''
                            INSERT INTO courses (name, acronym, href, department_id, category, syllabus_category, syllabus_href, syllabus_name) values (?, ?, ?, ?, ?, ?, ?, ?)
                        ''', (c.name, c.acronym, c.href, department_id, 'Available Syllabi', syllabus_category, syllabus_href, syllabus_name))
                        conn.commit()

In [ ]:
sr.semesters[0].departments[1].courses['Available Syllabi'][0]

<bound method Course.analyze of 60105-A: Cultural History of the Visual Arts>